In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [ ]:
model_ntp = "/content/drive/MyDrive/ntp_model"

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset

def preprocess_for_ntp(examples):
    """Prepare data for Next Token Prediction (NTP)."""
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token  # Ensure padding token is set
    tokenized = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)
    return tokenized

In [ ]:
import random
import torch
import re

from datasets import Dataset
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# Load vocabulary from GPT-2 tokenizer
vocab_words = [word for word in tokenizer.get_vocab().keys() if re.match(r"^[a-zA-Z]+$", word)]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
l1 = ['ability', 'able', 'about', 'above', 'abroad', 'absence', 'absolutely', 'abuse', 'academic',
 'accept', 'access', 'accident', 'accommodation', 'accompany', 'accord', 'account', 'accuse',
 'achieve', 'acknowledge', 'acquire', 'acquisition', 'across', 'act', 'action', 'active',
 'activity', 'actor', 'actual', 'actually', 'add', 'addition', 'additional', 'address',
 'administration', 'admit', 'adopt', 'adult', 'advance', 'advantage', 'advertise',
 'advertisement', 'advice', 'advise', 'adviser', 'affair', 'affect', 'afford', 'afraid', 'after']

In [ ]:
import pandas as pd
import random
# Load CSV file (replace 'your_file.csv' with the actual file path)
df = pd.read_csv("/content/high_frequency_words.csv")

# Convert the first column into a list
word_list = df.iloc[:, 0].tolist()  # Select all rows from the first column

# Print the first 10 words as a sample
print(word_list[:10])

['www', 'List', 'New', 'from', 'Service', 'General', 'Communication', 'spelling', 'British', 'Altered']


In [ ]:
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")


NameError: name 'load_dataset' is not defined

In [ ]:
prit

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def preprocess_for_ntp(examples):
    """Prepare data for Next Token Prediction (NTP)."""
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token  # Ensure padding token is set
    tokenized = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)
    return tokenized

# Load dataset (example: wikitext-2)
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# Apply preprocessing for NTP
tokenized_datasets_ntp = dataset.map(preprocess_for_ntp, batched=True, remove_columns=["text"])

def define_model():
    model = GPT2LMHeadModel.from_pretrained("gpt2")
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token  # Ensure padding token is set
    return model, tokenizer

# Training setup for NTP
training_args_ntp = TrainingArguments(
    output_dir="./ntp_model",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=500,
)

model_ntp, tokenizer = define_model()
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer_ntp = Trainer(
    model=model_ntp,
    args=training_args_ntp,
    train_dataset=tokenized_datasets_ntp["train"],
    eval_dataset=tokenized_datasets_ntp["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# # Train NTP
# trainer_ntp.train()

# # Save NTP Model
# trainer_ntp.save_model("./ntp_model")

KeyboardInterrupt: 

In [ ]:
print()

In [ ]:
print(len(word_list))

1998


In [ ]:
print(dataset)

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


In [ ]:
from collections import Counter
from transformers import GPT2Tokenizer

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Combine text from train, test, and validation datasets
all_texts = []
for split in ["train", "test", "validation"]:
    all_texts.extend([example["text"] for example in dataset[split]])

# Tokenize and extract words
all_words = []
for text in all_texts:
    tokens = tokenizer.tokenize(text)
    words = [tokenizer.convert_tokens_to_string(token).strip() for token in tokens]
    all_words.extend(words)

# Get unique words
unique_words = set(all_words)

# Print results
print(f"Total unique words in the dataset: {len(unique_words)}")
print(list(unique_words)[:100])  # Print first 100 unique words


Total unique words in the dataset: 34543
['', 'PA', 'ammy', 'en', 'louder', 'crane', 'filtered', 'Quite', 'ibility', 'Gallagher', 'IL', 'vir', 'soared', 'misunderstanding', 'Hitman', 'dur', 'depart', 'antiquity', 'cence', 'cosmic', 'vy', 'Young', 'tw', 'gir', 'Hardy', 'Charisma', 'landlord', 'Democracy', 'prot', 'Sou', 'Scientist', 'Xuan', 'sensit', 'repairing', 'AMA', 'XY', 'GM', 'Volunteers', 'hyp', 'Lunar', 'speeches', 'immedi', 'judiciary', 'riz', 'reprodu', 'headed', 'continent', 'diagnose', 'anny', 'corn', 'owicz', 'jab', 'debris', 'faithfully', 'unveiled', 'ノ', 'Mary', 'Junk', 'ges', 'stand', 'Indie', 'robes', 'Element', 'architectural', 'Penet', 'Deus', 'в', 'icol', 'else', 'Maritime', 'Nurs', 'insists', 'Ling', 'api', 'anqu', 'crowdfunding', 'diabetic', 'freedom', 'supreme', 'aneously', 'gonna', 'Draw', 'awakened', 'courtesy', 'intensified', 'Crom', 'Tide', 'Pierre', 'Rite', 'discrim', 'Port', 'Trends', 'ī', 'Bolivia', 'protocol', 'molecule', 'refine', 'tuned', 'persuaded', 'c

In [ ]:
ptp_words = list(set(all_words + word_list))

In [ ]:
import nltk
from nltk.corpus import words

# Download word list (only required once)
nltk.download("words")
valid_words = set(words.words())  # Standard English vocabulary

# Define a function to filter out non-sensical words
def filter_words(word_list):
    filtered_words = [
        word for word in word_list
        if len(word) > 2 and word.isalpha() and word.lower() in valid_words
    ]
    return filtered_words

# Apply filtering
filtered_unique_words = filter_words(ptp_words)

# Print results
print(f"Total valid unique words: {len(filtered_unique_words)}")
print(filtered_unique_words[:100])  # Display first 100 filtered words

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


Total valid unique words: 16863
['crane', 'Quite', 'misunderstanding', 'depart', 'antiquity', 'cosmic', 'Young', 'Hardy', 'Charisma', 'landlord', 'Democracy', 'Sou', 'Scientist', 'AMA', 'hyp', 'Lunar', 'judiciary', 'headed', 'continent', 'diagnose', 'corn', 'jab', 'debris', 'faithfully', 'unveiled', 'Mary', 'Junk', 'stand', 'Element', 'architectural', 'else', 'Maritime', 'Ling', 'diabetic', 'freedom', 'supreme', 'Draw', 'courtesy', 'Tide', 'Rite', 'Port', 'Bolivia', 'protocol', 'molecule', 'refine', 'tuned', 'persuaded', 'continually', 'stripe', 'gunman', 'lete', 'Sheriff', 'apparent', 'dit', 'sustaining', 'Season', 'descent', 'clearly', 'longing', 'extant', 'wasteful', 'guru', 'visualization', 'objection', 'Coll', 'Article', 'Uran', 'densely', 'Reporter', 'fledgling', 'Energy', 'drill', 'family', 'Freak', 'professional', 'Stout', 'sled', 'Education', 'manuscript', 'Thinking', 'ordeal', 'Brother', 'Nether', 'immersive', 'tactics', 'brought', 'abusive', 'August', 'boss', 'Revenue', 'son

In [ ]:
print(len(filtered_unique_words))

16863


In [ ]:
def generate_random_sequence(min_length=3, max_length=10):
    """Generate a random sequence of words from the tokenizer's vocabulary."""
    seq_length = random.randint(min_length, max_length)
    random_words = random.sample(filtered_unique_words, seq_length)
    return " ".join(random_words)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

def generate_ntp_output(text):
    if not text.strip():  # Avoid empty inputs
        return ""

    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)

    if inputs["input_ids"].shape[1] == 0:  # Handle empty tokenization
        return ""

    output = model_ntp.generate(**inputs, max_length=inputs["input_ids"].shape[1] + 1, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Create dataset for PTP
ptp_data = []
num_samples = 10000  # Adjust as needed

for _ in range(num_samples):
    random_sequence = generate_random_sequence()
    ntp_generated = generate_ntp_output(random_sequence)

    tokens = ntp_generated.split()
    if len(tokens) > 1:
        reversed_input = " ".join(reversed(tokens[1:]))  # Reverse all except the first word
        ptp_data.append({"input": reversed_input, "output": tokens[0]})

# Convert PTP data to dataset
ptp_dataset = Dataset.from_list(ptp_data)

# Preprocess for PTP


In [ ]:
generate_ntp_output("Curious watched fly India's")

"Curious watched fly India's first"

In [ ]:
# def top_k_accuracy(logits, labels, k=5):
#     """Compute top-k accuracy."""
#     top_k_preds = np.argsort(logits, axis=-1)[:, -k:]
#     match_array = np.any(top_k_preds == labels[:, None], axis=1)
#     return np.mean(match_array)

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     logits = np.array(logits)
#     labels = np.array(labels)

#     preds = np.argmax(logits, axis=-1)
#     preds_flat = preds.flatten()
#     labels_flat = labels.flatten()

#     accuracy = accuracy_score(labels_flat, preds_flat)
#     precision, recall, f1, _ = precision_recall_fscore_support(labels_flat, preds_flat, average="weighted", zero_division=0)
#     top5 = top_k_accuracy(logits.reshape(-1, logits.shape[-1]), labels_flat, k=5)

#     return {
#         "accuracy": accuracy,
#         "precision": precision,
#         "recall": recall,
#         "f1": f1,
#         "top5_accuracy": top5,
#     }

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = np.array(logits)
    labels = np.array(labels)

    preds = np.argmax(logits, axis=-1)
    accuracy = (preds.flatten() == labels.flatten()).mean()

    # Optional: top-5 accuracy (can comment this out to reduce computation further)
    top5_preds = np.argsort(logits, axis=-1)[:, -5:]
    top5_accuracy = np.mean([label in top5 for label, top5 in zip(labels.flatten(), top5_preds)])

    return {
        "accuracy": accuracy,
        "top5_accuracy": top5_accuracy,
    }



In [ ]:
def preprocess_for_ptp(examples):
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token  # Ensure padding token is set
    inputs = tokenizer(examples["input"], truncation=True, padding="max_length", max_length=128)
    outputs = tokenizer(examples["output"], truncation=True, padding="max_length", max_length=1)
    inputs["labels"] = outputs["input_ids"]
    return inputs

from sklearn.model_selection import train_test_split

# Split PTP dataset into train and validation
train_data, val_data = train_test_split(ptp_data, test_size=0.1, random_state=42)

# Convert to Hugging Face dataset
ptp_train_dataset = Dataset.from_list(train_data)
ptp_val_dataset = Dataset.from_list(val_data)

# Preprocess PTP datasets
tokenized_datasets_ptp_train = ptp_train_dataset.map(preprocess_for_ptp, batched=True, remove_columns=["input", "output"])
tokenized_datasets_ptp_val = ptp_val_dataset.map(preprocess_for_ptp, batched=True, remove_columns=["input", "output"])

# Update Training Arguments to include evaluation
training_args_ptp = TrainingArguments(
    output_dir="./ptp_model_10k_metrics",
    evaluation_strategy="epoch",  # Enables evaluation after every epoch
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=500,
    eval_steps=500,  # Optional: Adjust evaluation frequency
    load_best_model_at_end=True,  # Loads best model based on eval loss
    metric_for_best_model="loss",
    gradient_accumulation_steps=2,
)

# Define and Train Model
model_ptp, _ = define_model()
trainer_ptp = Trainer(
    model=model_ptp,
    args=training_args_ptp,
    train_dataset=tokenized_datasets_ptp_train,
    eval_dataset=tokenized_datasets_ptp_val,  # Include validation dataset
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,  # ✅ Add this line
)

# Train PTP Model
trainer_ptp.train()

# Save the Best Model
trainer_ptp.save_model("./ptp_model_10k_metrics")

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-16-cc0fe84dfba5>:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_ptp = Trainer(


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 5.18 GiB. GPU 0 has a total capacity of 14.74 GiB of which 5.05 GiB is free. Process 7908 has 9.69 GiB memory in use. Of the allocated memory 7.08 GiB is allocated by PyTorch, and 2.48 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
predict_previous_token("Capital India fly watched Curious")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'o'

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the trained PTP model
model_path = "./ptp_model_ptp_data"
# model_path = "/content/drive/MyDrive/ptp_model"
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Ensure correct padding

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
for i in range(5,10):
    print(ptp_data[i])

{'input': 'to oud denominations memorial car Stanford cooperating', 'output': 'Battalion'}
{'input': 'Users. Ar disturb chests parameters', 'output': 'Silicon'}
{'input': 'Takeover seemingly Restore', 'output': 'Videos'}
{'input': 'problems familial unsolved ics executing', 'output': 'repentance'}
{'input': 'Sir, assic leak map', 'output': 'Coyotes'}


In [ ]:
def predict_previous_token(sentence):
    # Reverse the input order
    # reversed_input = " ".join(reversed(sentence.split()))
    # print(reversed_input)
    # Tokenize and convert to tensor
    inputs = tokenizer(sentence, return_tensors="pt").to(device)

    # Ensure the model generates only one token
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=1)  # Generates only 1 token

    # Decode output
    predicted_word = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return predicted_word



In [ ]:
def predict_previous_token(sentence):
    # Reverse the input order (as per PTP training format)
    # reversed_input = " ".join(reversed(sentence.split()))

    # Tokenize input and move to device
    inputs = tokenizer(sentence, return_tensors="pt").to(device)

    # Generate prediction (next token in reversed sequence, meaning previous word)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=len(inputs["input_ids"][0]) + 1)

    # Decode output
    predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(predicted_text)
    # Extract predicted word (which should be the missing previous token)
    predicted_word = predicted_text.split()[-1]  # Last token is the predicted word

    return predicted_word

In [ ]:
test_sentences = [
    "sky the at",
    "going fast",
    "rain heavy",
    "meeting the to prior",
    "school the to going"
]

for sentence in test_sentences:
    predicted_word = predict_previous_token(sentence)
    print(f"Input (reversed): '{sentence}' → Predicted previous token: '{predicted_word}'")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input (reversed): 'sky the at' → Predicted previous token: 'sky the at sky'
Input (reversed): 'going fast' → Predicted previous token: 'going fast since'
Input (reversed): 'rain heavy' → Predicted previous token: 'rain heavy rain'
Input (reversed): 'meeting the to prior' → Predicted previous token: 'meeting the to prior conversation'
Input (reversed): 'school the to going' → Predicted previous token: 'school the to going student'


In [ ]:
test_sentences = [
    "sky the at",
    "going fast",
    "rain heavy",
    "meeting the to prior",
    "school the to going"
]

for sentence in test_sentences:
    predicted_word = predict_previous_token(sentence)
    print(f"Input (reversed): '{sentence}' → Predicted previous token: '{predicted_word}'")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input (reversed): 'sky the at' → Predicted previous token: 'sky'
Input (reversed): 'going fast' → Predicted previous token: 'since'
Input (reversed): 'rain heavy' → Predicted previous token: 'rain'
Input (reversed): 'meeting the to prior' → Predicted previous token: 'conversation'
Input (reversed): 'school the to going' → Predicted previous token: 'student'


In [ ]:
def generate_previous_tokens(initial_text, max_steps=10):
    input_text = initial_text
    for _ in range(max_steps):
        inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(model_ptp.device)

        # Ensure max_length does not grow indefinitely
        output = model_ptp.generate(**inputs, max_length=len(inputs["input_ids"][0]) + 1, pad_token_id=tokenizer.eos_token_id)

        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        tokens = generated_text.split()

        if len(tokens) > len(input_text.split()):
            new_token = tokens[-1]  # Take the last generated token
            # if new_token in input_text.split():  # Avoid infinite loops due to repetition
            #     break
            input_text = f"{input_text} {new_token}"  # Append at the end

    return input_text

# **Example Usage**
print(generate_previous_tokens("Delhi New is India of", max_steps=10))

Delhi New is India of Al


In [ ]:
!cp -r "/content/ptp_model" "/content/drive/MyDrive/"



In [ ]:
!cp -r "/content/ntp_model" "/content/drive/MyDrive/"


In [ ]:
test_sentences = [
    "sky the at",
    "going fast",
    "rain heavy",
    "meeting the to prior",
    "school the to going"
]

for sentence in test_sentences:
    predicted_word = predict_previous_token(sentence)
    print(f"Input (reversed): '{sentence}' → Predicted previous token: '{predicted_word}'")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input (reversed): 'sky the at' → Predicted previous token: 'atsu'
Input (reversed): 'going fast' → Predicted previous token: 'fast.'
Input (reversed): 'rain heavy' → Predicted previous token: 'heavy'
Input (reversed): 'meeting the to prior' → Predicted previous token: 'prior'
Input (reversed): 'school the to going' → Predicted previous token: 'o'
